##### Personalized model. features: processed aws Image rekongition features, Target: Tap_labels self recorded emotion

In [131]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
import math
import statistics 


###### Uncomment the below code according to requirement, below code contains test cases: default with all features, different baselines, aggregated data.

In [132]:
# Navdeep line
#df = pd.read_csv("augmented_merge_alluser.csv")
# my line
df = pd.read_csv("Augmented/Second_run/final_data/user_02.csv")
dfX=df
#default 95 features
#dropping target columns and alphanumeric columns.
#dfX = df.drop(['FileName'], axis = 'columns')
#dfX = df.drop(['NewTarget','Target','FileName','PredictedEmotion'], axis = 'columns') #target is aws output, newtarget is self-reported emotion



#baseline - relevant features 71
#dfX = df.drop(['FileName','PredictedEmotion','NewTarget','Target','Confidence','BoundingBoxWidth','BoundingBoxHeight','BoundingBoxLeft','BoundingBoxTop','AgeRangeLow','AgeRangeHigh','SmileConfidence','EyeglassesConfidence','SunglassesConfidence','GenderConfidence','BeardConfidence','MustacheConfidence','EyesOpenConfidence','MouthOpenConfidence','PoseRoll','PoseYaw','PosePitch','QualityBrightness','QualitySharpness','BeardValue_ False','BeardValue_ True','MustacheValue_ False','MustacheValue_ True'], axis = 'columns') 

#baseline - eyes & eyebrows 45
#dfX = df.drop(['FileName','PredictedEmotion','NewTarget','Target','Confidence','BoundingBoxWidth','BoundingBoxHeight','BoundingBoxLeft','BoundingBoxTop','AgeRangeLow','AgeRangeHigh','SmileConfidence','EyeglassesConfidence','SunglassesConfidence','GenderConfidence','BeardConfidence','MustacheConfidence','EyesOpenConfidence','MouthOpenConfidence','PoseRoll','PoseYaw','PosePitch','QualityBrightness','QualitySharpness','BeardValue_ False','BeardValue_ True','MustacheValue_ False','MustacheValue_ True','upperJawlineLeftX', 'upperJawlineLeftY','midJawlineLeftX', 'midJawlineLeftY', 'chinBottomX', 'chinBottomY',
#       'midJawlineRightX', 'midJawlineRightY', 'upperJawlineRightX',
#       'upperJawlineRightY', 'MouthOpenValue_ False', 'MouthOpenValue_ True','mouthLeftX', 'mouthLeftY', 'mouthRightX', 'mouthRightY', 'noseX',
#       'noseY','noseLeftX', 'noseLeftY', 'noseRightX', 'noseRightY', 'mouthUpX',
#       'mouthUpY', 'mouthDownX', 'mouthDownY'], axis = 'columns') 

#baseline - mouth 19
#dfX = df.drop(['NewTarget','Target','Confidence','BoundingBoxWidth','BoundingBoxHeight','BoundingBoxLeft','BoundingBoxTop','AgeRangeLow','AgeRangeHigh','SmileConfidence','EyeglassesConfidence','SunglassesConfidence','GenderConfidence','BeardConfidence','MustacheConfidence','EyesOpenConfidence','MouthOpenConfidence','PoseRoll','PoseYaw','PosePitch','QualityBrightness','QualitySharpness','BeardValue_ False','BeardValue_ True','MustacheValue_ False','MustacheValue_ True','eyeLeftX', 'eyeLeftY', 'eyeRightX', 'eyeRightY','noseX',
#       'noseY', 'leftEyeBrowLeftX', 'leftEyeBrowLeftY', 'leftEyeBrowRightX',
#       'leftEyeBrowRightY', 'leftEyeBrowUpX', 'leftEyeBrowUpY',
#       'rightEyeBrowLeftX', 'rightEyeBrowLeftY', 'rightEyeBrowRightX',
#       'rightEyeBrowRightY', 'rightEyeBrowupX', 'rightEyeBrowupY',
#       'leftEyeLeftX', 'leftEyeLeftY', 'leftEyeRightX', 'leftEyeRightY',
#       'leftEyeUpX', 'leftEyeUpY', 'leftEyeDownX', 'leftEyeDownY',
#       'rightEyeLeftX', 'rightEyeLeftY', 'rightEyeRightX', 'rightEyeRightY',
#       'rightEyeUpX', 'rightEyeUpY', 'rightEyeDownX', 'rightEyeDownY',
#       'noseLeftX', 'noseLeftY', 'noseRightX', 'noseRightY','leftPupilX', 'leftPupilY',
#       'rightPupilX', 'rightPupilY', 'upperJawlineLeftX', 'upperJawlineLeftY',
#       'midJawlineLeftX', 'midJawlineLeftY', 'chinBottomX', 'chinBottomY',
#       'midJawlineRightX', 'midJawlineRightY', 'upperJawlineRightX',
#       'upperJawlineRightY'], axis = 'columns') 


# #baseline - eyes 33
# dfX = df.drop(['NewTarget','Target','Confidence','BoundingBoxWidth','BoundingBoxHeight','BoundingBoxLeft','BoundingBoxTop','AgeRangeLow','AgeRangeHigh','SmileConfidence','EyeglassesConfidence','SunglassesConfidence','GenderConfidence','BeardConfidence','MustacheConfidence','EyesOpenConfidence','MouthOpenConfidence','PoseRoll','PoseYaw','PosePitch','QualityBrightness','QualitySharpness','BeardValue_ False','BeardValue_ True','MustacheValue_ False','MustacheValue_ True','upperJawlineLeftX', 'upperJawlineLeftY','midJawlineLeftX', 'midJawlineLeftY', 'chinBottomX', 'chinBottomY',
#        'midJawlineRightX', 'midJawlineRightY', 'upperJawlineRightX',
#        'upperJawlineRightY', 'MouthOpenValue_ False', 'MouthOpenValue_ True','mouthLeftX', 'mouthLeftY', 'mouthRightX', 'mouthRightY', 'noseX',
#        'noseY','noseLeftX', 'noseLeftY', 'noseRightX', 'noseRightY', 'mouthUpX',
#        'mouthUpY', 'mouthDownX', 'mouthDownY','leftEyeBrowLeftX', 'leftEyeBrowLeftY', 'leftEyeBrowRightX',
#        'leftEyeBrowRightY', 'leftEyeBrowUpX', 'leftEyeBrowUpY',
#        'rightEyeBrowLeftX', 'rightEyeBrowLeftY', 'rightEyeBrowRightX',
#        'rightEyeBrowRightY', 'rightEyeBrowupX', 'rightEyeBrowupY'], axis = 'columns') 

# for all users merge data, drop encoded features as encoded columns are not consistent for every user.
#dfX = df.drop(['NewTarget','Target','FileName','PredictedEmotion','BeardValue_ False','BeardValue_ True','EyeglassesValue_ False','EyeglassesValue_ True','EyesOpenValue_ False','EyesOpenValue_ True',"GenderValue_ 'Female'","GenderValue_ 'Male'",'MouthOpenValue_ False','MouthOpenValue_ True','MustacheValue_ False','MustacheValue_ True','SmileValue_ False','SmileValue_ True','SunglassesValue_ False'], axis = 'columns')  


dfY = df['NewTarget']
#getting count of each emotion
total_count = 0
happy_count = 0
sad_count = 0
relaxed_count = 0
stressed_count  = 0
for value in dfY:
    if value =="Relaxed":
        total_count+=1
        relaxed_count+=1
    elif value =="Stressed":
        stressed_count+=1
        total_count+=1
    elif value =="Happy":
        happy_count += 1
        total_count+=1
    elif value =="Sad":
        sad_count +=1
        total_count+=1
        
print(happy_count, sad_count, stressed_count, relaxed_count)

print(total_count)
dfX

20 0 12 29
61


,NewTarget,Confidence,BoundingBoxWidth,BoundingBoxHeight,BoundingBoxLeft,BoundingBoxTop,AgeRangeLow,AgeRangeHigh,SmileConfidence,EyeglassesConfidence,...,QualitySharpness,SmileValue_ False,EyeglassesValue_ False,SunglassesValue_ False,GenderValue_ 'Male',BeardValue_ False,MustacheValue_ False,EyesOpenValue_ False,EyesOpenValue_ True,MouthOpenValue_ False
0,Relaxed,99.998215,0.421724,0.454994,0.564394,0.296910,29,45,95.905037,99.325333,...,73.322098,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
1,Relaxed,99.997833,0.487869,0.431044,0.025687,0.192112,39,57,95.974045,97.867676,...,67.227310,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
2,Relaxed,99.997177,0.380385,0.430185,0.591897,0.285235,35,51,89.244354,67.642685,...,32.208035,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
3,Happy,99.999870,0.671072,0.768130,0.375549,0.070991,13,23,96.907982,97.947769,...,38.896011,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
4,Happy,99.999252,0.741767,0.815951,0.077193,0.070835,13,23,83.606628,97.043182,...,60.490417,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Relaxed,90.265076,0.401262,0.481581,0.065480,0.177095,22,34,97.666862,80.592644,...,94.082626,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
57,Stressed,99.996040,0.631752,0.521891,0.025617,0.291062,13,23,98.414078,98.351112,...,38.896011,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
58,Stressed,99.999084,0.675894,0.522164,0.049374,0.281875,12,22,98.998306,98.788956,...,78.643501,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
59,Stressed,99.991295,0.716915,0.457849,0.133498,0.042515,22,34,97.024086,98.615303,...,60.490417,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0


In [133]:
#df_f = dfX[dfX['NewTarget'] != -99]  # discard no response
# this part is for swapping two columns
'''
cols = list(df_f.columns)
a, b = cols.index('Confidence'), cols.index('NewTarget')
cols[b], cols[a] = cols[a], cols[b]
df_f = df_f[cols]
df_f=df_f.drop(['NewTarget'],axis=1)
'''
df_f=dfX
l=df_f.columns.values
feature_indices=[0,1,5,6,7,8,9,10,11,12,14,77,78]# attribute indices
features=[]
for i in feature_indices:
    features.append(l[i])
#print(features)    
df_f=df_f[features]
dfY = df['NewTarget']
df_f=df_f.drop(['NewTarget'],axis=1)

df_f

,Confidence,BoundingBoxTop,AgeRangeLow,AgeRangeHigh,SmileConfidence,EyeglassesConfidence,SunglassesConfidence,GenderConfidence,BeardConfidence,EyesOpenConfidence,PoseYaw,PosePitch
0,99.998215,0.296910,29,45,95.905037,99.325333,54.037674,98.830338,97.813850,99.933891,5.443866,22.575872
1,99.997833,0.192112,39,57,95.974045,97.867676,92.354645,80.953369,92.345337,99.489326,-3.998221,21.736629
2,99.997177,0.285235,35,51,89.244354,67.642685,55.213390,65.148041,81.111664,98.470810,3.604582,36.535973
3,99.999870,0.070991,13,23,96.907982,97.947769,99.162178,89.953819,96.851501,72.130882,4.426538,3.211774
4,99.999252,0.070835,13,23,83.606628,97.043182,99.033180,96.341339,97.664467,64.733192,-4.995044,8.364226
...,...,...,...,...,...,...,...,...,...,...,...,...
56,90.265076,0.177095,22,34,97.666862,80.592644,88.453285,51.746559,68.557640,92.094490,-11.944880,20.391027
57,99.996040,0.291062,13,23,98.414078,98.351112,99.226852,98.766266,98.406509,98.710159,3.363793,21.702749
58,99.999084,0.281875,12,22,98.998306,98.788956,99.514679,98.949951,98.456161,54.334599,0.937859,18.721081
59,99.991295,0.042515,22,34,97.024086,98.615303,99.518196,99.335640,98.535690,71.365273,-1.214663,18.978699


In [134]:
# Binarize the output
#dfY = label_binarize(dfY, classes=[2, -2, 1, 0])
dfY = label_binarize(dfY, classes=["Happy","Sad","Stressed","Relaxed"])
dfY

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0,

In [135]:
n_classes=dfY.shape[1]

In [136]:
# implementing k fold cross validation
k = 5
from sklearn.model_selection import KFold
kf = KFold(n_splits=k,shuffle = True)

In [137]:
happy_list = []
sad_list = []
relaxed_list = []
stressed_list = []
roc_auc_avg = []
df_f = df_f.values


In [138]:
#model = RandomForestClassifier(n_estimators = 50)

In [139]:
# for each split
fold=0
for train_index, test_index in kf.split(df_f):
    fold+=1
    print("fold",fold)
    
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df_f[train_index], df_f[test_index]
    print("Train size=",X_train.shape[0])
    print("Test size=",X_test.shape[0])
    y_train, y_test = dfY[train_index], dfY[test_index]
    model = OneVsRestClassifier(RandomForestClassifier(n_estimators=50))
    Y_prob=model.fit(X_train,y_train).predict_proba(X_test)
    #y_score = model.score(X_test,y_test)
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], Y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
        
        if(math.isnan(roc_auc[i])):
            continue
        elif(i==0):
            happy_list.append(roc_auc[i])
            print("Happy=",roc_auc[i])            
        elif(i==1):
            sad_list.append(roc_auc[i])
            print("sad=",roc_auc[i])            
        elif(i==2):
            stressed_list.append(roc_auc[i])
            print("stressed=",roc_auc[i])            
        else:
            relaxed_list.append(roc_auc[i])
            print("relaxed=",roc_auc[i])
     

fold 1
Train size= 48
Test size= 13


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))


Happy= 1.0
stressed= 1.0
relaxed= 1.0
fold 2
Train size= 49
Test size= 12


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))


Happy= 0.8611111111111112
relaxed= 1.0
fold 3
Train size= 49
Test size= 12


...

/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))


Happy= 0.962962962962963
stressed= 1.0
relaxed= 1.0
fold 5
Train size= 49
Test size= 12


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))


Happy= 0.9259259259259259
stressed= 1.0
relaxed= 1.0


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


In [140]:
#Emotion wise Score
happy_score,sad_score,stress_score,relax_score=0,0,0,0
if(len(happy_list)!=0):
    happy_score=statistics.mean(happy_list)
    print("happy score=",happy_score)    
if(len(sad_list)!=0):
    sad_score=statistics.mean(sad_list)
    print("sad_score=",sad_score)    
if(len(stressed_list)!=0):
    stress_score=statistics.mean(stressed_list)
    print("stressed_score=",stress_score)    
if(len(relaxed_list)!=0):    
    relax_score=statistics.mean(relaxed_list)
    print("relax_score=",relax_score)  

happy score= 0.9388888888888889
stressed_score= 1.0
relax_score= 1.0


In [141]:
#user weighted score
user_weighted_avg_score = 0
user_weighted_avg_score = (happy_score * happy_count + sad_score* sad_count + relax_score * relaxed_count + stress_score * stressed_count)/total_count
user_weighted_avg_score

0.9799635701275046